#**1: Install All the Required Packages**

In [48]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install tiktoken

In [49]:
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.8 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

#**2: Import All the Required Libraries**

In [1]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#Create an Environment

In [4]:
os.environ['OPENAI_API_KEY'] = API_KEY

#**3: Pass the URLs and extract the data from these URLs**

In [5]:
URLs=[
    'https://id.wikipedia.org/wiki/JKT48'
]

In [6]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [7]:
data

[Document(metadata={'source': 'https://id.wikipedia.org/wiki/JKT48'}, page_content='JKT48\n\nBasa Bali\n\nBanjar\n\nCebuano\n\nEnglish\n\nEspañol\n\nFrançais\n\nItaliano\n\n日本語\n\nJawa\n\n한국어\n\nBahasa Melayu\n\nPolski\n\nPortuguês\n\nRomână\n\nไทย\n\nTagalog\n\nTiếng Việt\n\nWinaray\n\n中文\n\n文言\n\nSunting pranala\n\nDari Wikipedia bahasa Indonesia, ensiklopedia bebas\n\nJKT48 Logo JKT48 (sejak 2011) Informasi latar belakang Asal Jakarta , Indonesia Genre Indo pop J-pop pop pop remaja bubblegum pop pop rock dangdut jazz ska Tahun aktif ( 2011-12-17 ) 17 Desember 2011 – sekarang (12 tahun) Label Studio rekaman Dentsu Inter Admark (2011–2022) HITS Records (label fisik, 2012–2020) [ a ] Indonesia Musik Nusantara (2022–) AKA Virtual (khusus JKT48V , 2023–) Lisensi Vernalossom (2011–) Superball (2023–) Distributor (Publisher) Sony Music Entertainment Indonesia (2013–2018, uncredited ) Universal Music Indonesia (2015–2020, uncredited ) Massive Music Entertainment (2019-2021, uncredited ) INS

In [8]:
len(data)

1

#**4: Split the Text into Chunks**

In [9]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [10]:
text_chunks=text_splitter.split_documents(data)

In [11]:
len(text_chunks)

104

In [12]:
text_chunks[0]

Document(metadata={'source': 'https://id.wikipedia.org/wiki/JKT48'}, page_content='JKT48\nBasa Bali\nBanjar\nCebuano\nEnglish\nEspañol\nFrançais\nItaliano\n日本語\nJawa\n한국어\nBahasa Melayu\nPolski\nPortuguês\nRomână\nไทย\nTagalog\nTiếng Việt\nWinaray\n中文\n文言\nSunting pranala\nDari Wikipedia bahasa Indonesia, ensiklopedia bebas')

In [13]:
text_chunks[1]

Document(metadata={'source': 'https://id.wikipedia.org/wiki/JKT48'}, page_content='JKT48 Logo JKT48 (sejak 2011) Informasi latar belakang Asal Jakarta , Indonesia Genre Indo pop J-pop pop pop remaja bubblegum pop pop rock dangdut jazz ska Tahun aktif ( 2011-12-17 ) 17 Desember 2011 – sekarang (12 tahun) Label Studio rekaman Dentsu Inter Admark (2011–2022) HITS Records (label fisik, 2012–2020) [ a ] Indonesia Musik Nusantara (2022–) AKA Virtual (khusus JKT48V , 2023–) Lisensi Vernalossom (2011–) Superball (2023–) Distributor (Publisher) Sony Music Entertainment Indonesia (2013–2018, uncredited ) Universal Music Indonesia (2015–2020, uncredited ) Massive Music Entertainment (2019-2021, uncredited ) INSIDE (distributor digital, 2016-2022) Believe Music (distributor digital, sejak pertengahan 2010an) Jagonya Musik & Sport Indonesia (distributor khusus milik KFC , 2017) Artis terkait AKB48 Group Sub-Grup (bubar) 4 Gulali JKT48 Dangdut JKT48 Band JKT48 Dance Project JKT48 Acoustic Tim Olahra

In [14]:
text_chunks[2]

Document(metadata={'source': 'https://id.wikipedia.org/wiki/JKT48'}, page_content='Logo JKT48 NEW ERA (sejak Juni 2022) Nama dagang JKT48 Sebelumnya PT Dentsu Inter Admark Media Group Indonesia (2011–2022) (melalui JKT48 Project (2011–2018) dan DentsuX Entertainment (2018–2022) ) Jenis Perseroan terbatas Industri musik dan hiburan Genre musik , tari , dan hiburan Didirikan 11 September 2011 ; 13 tahun lalu ( 2011-09-11 ) (pertama kali diumumkan) 2 November 2011 ; 12 tahun lalu ( 2011-11-02 ) (generasi pertama diumumkan) 17 Desember 2011 ; 12 tahun lalu ( 2011-12-17 ) (debut secara perdana) 8 September 2012 ; 12 tahun lalu ( 2012-09-08 ) ( Teater JKT48 ) Pendiri Yasushi Akimoto Harris Thajeb Reino Barack [ a ] Kantor pusat JKT48 Office @ IDN HQ Lantai 3 Menara Global , Jl. Jenderal Gatot Subroto Kav. 27, Kuningan Timur , Setiabudi , Jakarta Selatan 12950, Jakarta , Indonesia Cabang JKT48 Theater Lantai 4 fX Sudirman , Jl. Jenderal Sudirman No. 1, RT .1/ RW .3, Gelora , Tanah Abang , Jak

#**5: Download the OpenAI Embeddings or Hugging Face Embeddings**

In [15]:
embeddings = OpenAIEmbeddings()

<ipython-input-15-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
embeddings

In [17]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [18]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

#**07: Create a Large Language Model (LLM) Wrapper**

In [19]:
llm=ChatOpenAI()

<ipython-input-19-bd12acb39de2>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI()


In [ ]:
llm

In [22]:
llm.predict("apa itu pytagoras")

'Pythagoras adalah seorang filsuf dan matematikawan kuno dari Yunani yang terkenal dengan teorema Pythagoras, yang menyatakan bahwa dalam segitiga siku-siku, kuadrat panjang sisi miring sama dengan jumlah kuadrat panjang sisi yang lain. Pythagoras juga dikenal karena kontribusinya dalam bidang musik, astronomi, dan filosofi. Ia juga mendirikan sekolah filsafat dan matematika di Kroton, Italia, yang dikenal sebagai sekolah Pythagoras.'

#**08: Initialize the Retrieval QA with Source Chain**

In [23]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [24]:
result=chain({"question": "Kapan JKT48 berdiri?"}, return_only_outputs=True)

<ipython-input-24-5614aaa15836>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"question": "Kapan JKT48 berdiri?"}, return_only_outputs=True)
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/openai.py:173: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [25]:
result['answer']

'JKT48 didirikan pada tanggal 3 Desember 2011.\n'

In [26]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [27]:
wrapped_text

'JKT48 didirikan pada tanggal 3 Desember 2011.'

In [32]:
result=chain({"question": "Apa konsep yang dibawa jkt48?"}, return_only_outputs=True)

/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/openai.py:173: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [33]:
result['answer']

'Konsep yang dibawa oleh JKT48 adalah sebagai grup idola yang dipromosikan oleh perusahaan-perusahaan Jepang di Indonesia untuk menjadi bintang iklan produk dan layanan mereka. Mereka tampil dalam berbagai acara promosi dan konser bersama dengan AKB48.\n'

In [34]:
wrapped_text = textwrap.fill(result['answer'], width=100)

In [35]:
wrapped_text

'Konsep yang dibawa oleh JKT48 adalah sebagai grup idola yang dipromosikan oleh perusahaan-perusahaan\nJepang di Indonesia untuk menjadi bintang iklan produk dan layanan mereka. Mereka tampil dalam\nberbagai acara promosi dan konser bersama dengan AKB48.'

In [37]:
while True:
  query=input(f"Prompt: ")
  if query == 'exit':
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])


Prompt: berapa total member jkt?


/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/openai.py:173: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


Answer: JKT48 memiliki total 52 anggota.

Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
